In [1]:
import os
import pandas as pd

version = "v01"
main_publisher = 'MITOCW'
script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_chapters = pd.read_csv(path + 'chapters.csv', sep = ',')
df_chapters.head()

,Cid,Eid,Sid,Title,Text,BeginTimestamp,EndTimestamp,Corpus
0,0,0,0,"Lecture 1: Motivation, Intuition, and Examples",[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:59:36,MIT OpenCourseWare
1,1,1,0,Lecture 2: General Theory Part 0,[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:21:17,MIT OpenCourseWare
2,2,1,0,Lecture 2: General Theory Part 1,"I know that most of them-- in fact, infinitely...",0:21:44,0:43:37,MIT OpenCourseWare
3,3,1,0,Lecture 2: General Theory Part 2,You can replace R with just any metric space. ...,0:43:40,1:18:27,MIT OpenCourseWare
4,4,2,0,Lecture 3: Compact Sets in Rⁿ Part 0,[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:21:20,MIT OpenCourseWare


In [2]:
len(df_chapters)

1857

In [4]:
from concepts import text2RDF

df_chapters[154:].apply(lambda x : text2RDF(x['Text'], "../Output/Graphs/"+ version + "/"+ main_publisher + "/" + str(x['Cid'])), axis = 1)

Wikification for : 154
Graph generation for : 154
Saved in  ../Output/Graphs/v01/MITOCW/154.ttl
Wikification for : 155
Graph generation for : 155
Saved in  ../Output/Graphs/v01/MITOCW/155.ttl
Wikification for : 156
Graph generation for : 156
Saved in  ../Output/Graphs/v01/MITOCW/156.ttl
Wikification for : 157
Graph generation for : 157
Saved in  ../Output/Graphs/v01/MITOCW/157.ttl
Wikification for : 158
Graph generation for : 158
Saved in  ../Output/Graphs/v01/MITOCW/158.ttl
Wikification for : 159
Graph generation for : 159
Saved in  ../Output/Graphs/v01/MITOCW/159.ttl
Wikification for : 160
Graph generation for : 160
Saved in  ../Output/Graphs/v01/MITOCW/160.ttl
Wikification for : 161
Graph generation for : 161
Saved in  ../Output/Graphs/v01/MITOCW/161.ttl
Wikification for : 162
Graph generation for : 162
Saved in  ../Output/Graphs/v01/MITOCW/162.ttl
Wikification for : 163
Graph generation for : 163
Saved in  ../Output/Graphs/v01/MITOCW/163.ttl
Wikification for : 164
Graph generation 

154     ((http://en.wikipedia.org/wiki/Singular_they, ...
155     ((http://example.org/Chapter, http://purl.org/...
156     ((http://example.org/Chapter, http://purl.org/...
157     ((http://example.org/Chapter, http://purl.org/...
158     ((http://example.org/Chapter, http://purl.org/...
                              ...                        
1852    ((http://example.org/Chapter, http://purl.org/...
1853    ((http://en.wikipedia.org/wiki/Tao, https://un...
1854    ((http://en.wikipedia.org/wiki/How_(TV_series)...
1855    ((http://en.wikipedia.org/wiki/Multiverse, htt...
1856    ((http://en.wikipedia.org/wiki/Form_and_conten...
Length: 1703, dtype: object